## DNA 서열 관리 프로그램
---
* 이 프로그램의 실행에는 Biopython 모듈과 cx_Oracle의 설치가 필요합니다.
---
* 상단 메뉴의 Kernel - Restart & Rul All을 눌러 프로그램을 초기화한 상태로 실행 가능합니다.
* NCBI FASTA 형식을 그대로 붙여넣기하여 DB에 등록할 수 있습니다.

In [ ]:
#Biopython 모듈 설치
!pip install biopython

#cx_Oracle 모듈 설치
!pip install biopython

In [ ]:
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio.SeqUtils import GC
import cx_Oracle
import pandas as pd
from IPython.display import display

In [ ]:
def exit():
    print("\n--------------------------")
    print("프로그램 종료")

In [ ]:
def DNA_edit():    
    DNA = input("편집할 DNA 서열을 입력하세요: ").replace("N","")
    DNA = Seq(''.join(DNA.split(' ')),IUPAC.unambiguous_dna)

    while(1):
        print("\n--------------------------")
        print("1. DNA transcription")
        print("2. DNA translation")
        print("3. complement sequence")
        print("4. GC contents")
        print("5. 이전 메뉴로")
        print("9. 프로그램 종료")
        req = 0

        req = int(input("원하는 메뉴를 선택하세요: "))

        if(req == 1):
            mRNA = DNA.transcribe()
            print("\n--------------------------")
            print("변환된 mRNA 서열: "+mRNA)
        elif(req == 2):
            mRNA = DNA.transcribe()
            protein = mRNA.translate()
            print("\n--------------------------")
            print("변환된 아미노산 서열: "+protein)
        elif(req == 3):
            DNA_comp = DNA.complement()
            print("\n--------------------------")
            print("상보서열: "+DNA_comp)
        elif(req == 4):
            GC_per = GC(DNA)
            print("\n--------------------------")
            print("GC contents: "+str(GC_per)+"%")
        elif(req == 5):
            print("\n--------------------------")
            main()
        elif(req == 9):
            exit()

In [ ]:
def DB_view():
    print("\n--------------------------")
    print("1. 신규 등록")
    print("2. DB 열기")
    print("3. DB 검색(일련번호)")
    print("4. DB 검색(키워드)")
    print("5. 이전 메뉴로")
    print("9. 프로그램 종료")
    req = 0

    req = int(input("원하는 메뉴를 선택하세요: "))

    if(req == 1):
        print("\n--------------------------")
        DNA = input("등록할 NCBI FASTA 파일(DNA)을 붙여넣기하세요: ")
        tag = DNA.split('>')[1:][0].split()[0]
        name = ' '.join(DNA.split('>')[1:][0].split(',')[0].split()[1:])
        DNA_seq = Seq(''.join(DNA.split('>')[1:][0].split(',')[-1].split()[2:]).replace("N",""),IUPAC.unambiguous_dna)
        mRNA = DNA_seq.transcribe()
        protein = mRNA.translate()
        GC_per = GC(DNA_seq)
        
        conn = cx_Oracle.connect('scott/tiger@localhost:1521/xe')

        cursor = conn.cursor()

        cursor.execute("insert into DNA values('%s','%s','%s','%s','%s',%d)" %(tag,name,str(DNA_seq),str(mRNA),str(protein),GC_per))
        cursor.execute("commit")

        cursor.close()
        conn.close()
        
        print("\n--------------------------")
        print("FASTA 파일을 DB에 등록하였습니다")
        
        DB_view()
        
    elif(req == 2):
        print("\n--------------------------")
        conn = cx_Oracle.connect('scott/tiger@localhost:1521/xe')

        display(pd.read_sql("SELECT * FROM DNA", conn))

        conn.close()
        
        DB_view()
        
    elif(req == 3):
        print("\n--------------------------")
        tag = input("검색할 gene의 NCBI 일련번호를 입력하세요: ")

        conn = cx_Oracle.connect('scott/tiger@localhost:1521/xe')
        
        print("\n--------------------------")
        
        
        display(pd.read_sql('select * from DNA where tag = \'%s\'' %tag, conn))

        conn.close()
        
        DB_view()
        
    elif(req == 4):
        print("\n--------------------------")
        tag = input("검색할 gene의 이름 일부를 입력하세요: ")

        conn = cx_Oracle.connect('scott/tiger@localhost:1521/xe')
        
        print("\n--------------------------")
        
        display(pd.read_sql("select * from DNA where name like '%%%s%%' " %tag, conn))

        conn.close()
        
        DB_view()
        
    elif(req == 5):
        print("\n--------------------------")
        main()
    elif(req == 9):
        exit()

In [ ]:
def main():
    print("1. DNA 서열 편집")
    print("2. DB 관리")
    print("3. 프로그램 종료")
    req = 0

    while(req == 0):
        req = int(input("원하는 메뉴를 선택하세요: "))

        if(req == 1):
            print("\n--------------------------")
            print("DNA 서열 편집")
            DNA_edit()
        elif(req == 2):
            DB_view()
        elif(req == 3):
            exit()

In [ ]:
#프로그램 실행
main()